Hello and welcome to my capstone project notebook.
This notebook will be used to showcase the work that I've done to finish this class.

In [1]:
from geopy.geocoders import Nominatim
import folium
# Import the necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# requests for fetching html of website
import requests

In [2]:
geolocator = Nominatim(timeout=10, user_agent = "sje.Capstone-Project")
# Read the CSV using the City of Chicago data
stations = pd.read_csv("https://data.cityofchicago.org/api/views/8pix-ypme/rows.csv?accessType=DOWNLOAD")
stations.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


only blue line

In [68]:
blue_line = stations[ stations['BLUE'] == True ]
blue_line = pd.DataFrame(blue_line)
blue_line.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
33,30002,W,Austin (Forest Pk-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,"(41.870851, -87.776812)"
34,30001,E,Austin (O'Hare-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,"(41.870851, -87.776812)"
37,30013,S,Belmont (O'Hare Branch) (Forest Pk-bound),Belmont,Belmont (Blue Line),40060,False,False,True,False,False,False,False,False,False,False,"(41.938132, -87.712359)"
38,30012,N,Belmont (O'Hare Branch) (O'Hare-bound),Belmont,Belmont (Blue Line),40060,False,False,True,False,False,False,False,False,False,False,"(41.938132, -87.712359)"
61,30188,W,Cicero (Forest Pk-bound),Cicero,Cicero (Blue Line),40970,False,False,True,False,False,False,False,False,False,False,"(41.871574, -87.745154)"


remove duplicates

In [69]:
blue_line.drop_duplicates(subset ="Location", inplace = True)
blue_line.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
33,30002,W,Austin (Forest Pk-bound),Austin,Austin (Blue Line),40010,False,False,True,False,False,False,False,False,False,False,"(41.870851, -87.776812)"
37,30013,S,Belmont (O'Hare Branch) (Forest Pk-bound),Belmont,Belmont (Blue Line),40060,False,False,True,False,False,False,False,False,False,False,"(41.938132, -87.712359)"
61,30188,W,Cicero (Forest Pk-bound),Cicero,Cicero (Blue Line),40970,False,False,True,False,False,False,False,False,False,False,"(41.871574, -87.745154)"
67,30374,S,Clark/Lake (Forest Pk-bound),Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...",40380,True,False,True,False,False,False,False,False,False,False,"(41.885737, -87.630886)"
69,30085,W,Clinton (Forest Pk-bound),Clinton,Clinton (Blue Line),40430,False,False,True,False,False,False,False,False,False,False,"(41.875539, -87.640984)"


In [70]:
blue_line['Latitude'], blue_line['Longitude'] = blue_line['Location'].str.split(' ', 1).str

<ipython-input-70-401db6cdd0c2>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  blue_line['Latitude'], blue_line['Longitude'] = blue_line['Location'].str.split(' ', 1).str


In [71]:
# Drop all the extra columns
blue_line.drop(['DIRECTION_ID', 'STATION_NAME', 'MAP_ID', 'STOP_NAME', 'ADA', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O'], axis = 1, inplace=True)
blue_line['STATION_DESCRIPTIVE_NAME'] = blue_line['STATION_DESCRIPTIVE_NAME'].str.replace('(Blue Line)', '', regex=False)
blue_line.rename(columns={'STATION_DESCRIPTIVE_NAME': 'Station'}, inplace=True)
blue_line.head()

,STOP_ID,Station,Location,Latitude,Longitude
33,30002,Austin,"(41.870851, -87.776812)","(41.870851,",-87.776812)
37,30013,Belmont,"(41.938132, -87.712359)","(41.938132,",-87.712359)
61,30188,Cicero,"(41.871574, -87.745154)","(41.871574,",-87.745154)
67,30374,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)","(41.885737,",-87.630886)
69,30085,Clinton,"(41.875539, -87.640984)","(41.875539,",-87.640984)


In [105]:
blue_line.set_index("STOP_ID", inplace=True)
blue_line.head()

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154
30374,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)",41.885737,-87.630886
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984


In [112]:
blue_line.at[30374,'Station'] = 'Clark/Lake'
#blue_line.loc[30374]['Station'] = 'Clark/Lake'

In [119]:
blue_line['Station'] = blue_line["Station"].str.replace('(Blue Line - O\'Hare Branch)', '(OH Branch)', regex=False)
blue_line['Station'] = blue_line["Station"].str.replace('(Blue Line - Forest Park Branch)', '(FB Branch)', regex=False)

In [120]:
blue_line

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154
30374,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984
30045,Cumberland,"(41.984246, -87.838028)",41.984246,-87.838028
30116,Damen,"(41.909744, -87.677437)",41.909744,-87.677437
30063,Division,"(41.903355, -87.666496)",41.903355,-87.666496
30076,Forest Park,"(41.874257, -87.817318)",41.874257,-87.817318


In [72]:
blue_line['Longitude'] = blue_line["Longitude"].str.replace(')', '', regex=False)
blue_line['Latitude'] = blue_line["Latitude"].str.replace('(', '', regex=False)
blue_line['Latitude'] = blue_line["Latitude"].str.replace(',', '', regex=False)
blue_line[["Longitude"]] = blue_line[["Longitude"]].astype("float")
blue_line[["Latitude"]] = blue_line[["Latitude"]].astype("float")
blue_line.head()

,STOP_ID,Station,Location,Latitude,Longitude
33,30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812
37,30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359
61,30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154
67,30374,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)",41.885737,-87.630886
69,30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984


In [8]:
# Let's get the basic location data for Chicago, just to get started
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="sje_capstone-project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


Add zipcodes to dataframe for use later

In [9]:
# This function strips the parenthesis from the 'Location' column and returns a zipcode string
def getzip (location_info):
    
    zipcode = geolocator.geocode(location_info[1:-1], addressdetails=True).raw['address']['postcode']

    return zipcode[0:5]

In [10]:
blue_line['Zipcode'] = blue_line['Location'].apply(getzip)
blue_line.head()

,STOP_ID,STATION_NAME,STATION_DESCRIPTIVE_NAME,Location,Latitude,Longitude,Zipcode
33,30002,Austin,Austin (Blue Line),"(41.870851, -87.776812)",41.870851,-87.776812,60130
37,30013,Belmont,Belmont (Blue Line),"(41.938132, -87.712359)",41.938132,-87.712359,60659
61,30188,Cicero,Cicero (Blue Line),"(41.871574, -87.745154)",41.871574,-87.745154,60644
67,30374,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)",41.885737,-87.630886,60696
69,30085,Clinton,Clinton (Blue Line),"(41.875539, -87.640984)",41.875539,-87.640984,60606


In [11]:
blue_line.head(10)

,STOP_ID,STATION_NAME,STATION_DESCRIPTIVE_NAME,Location,Latitude,Longitude,Zipcode
33,30002,Austin,Austin (Blue Line),"(41.870851, -87.776812)",41.870851,-87.776812,60130
37,30013,Belmont,Belmont (Blue Line),"(41.938132, -87.712359)",41.938132,-87.712359,60659
61,30188,Cicero,Cicero (Blue Line),"(41.871574, -87.745154)",41.871574,-87.745154,60644
67,30374,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)",41.885737,-87.630886,60696
69,30085,Clinton,Clinton (Blue Line),"(41.875539, -87.640984)",41.875539,-87.640984,60606
76,30045,Cumberland,Cumberland (Blue Line),"(41.984246, -87.838028)",41.984246,-87.838028,60631
80,30116,Damen,Damen (Blue Line),"(41.909744, -87.677437)",41.909744,-87.677437,60647
90,30063,Division,Division (Blue Line),"(41.903355, -87.666496)",41.903355,-87.666496,60642
91,30076,Forest Park,Forest Park (Blue Line),"(41.874257, -87.817318)",41.874257,-87.817318,60130
103,30096,Grand,Grand (Blue Line),"(41.891189, -87.647578)",41.891189,-87.647578,60661


In [12]:
# create map of Chciago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, station in zip(blue_line['Latitude'], blue_line['Longitude'], blue_line['STATION_NAME']):
    label = station
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [13]:

# create map of Chciago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, station in zip(blue_line['Latitude'], blue_line['Longitude'], blue_line['STATION_NAME']):
    label = station
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)
    folium.Circle([lat, lng], radius=600, color='red', fill=False).add_to(map_chicago)
  
    
map_chicago


# CL Scraping Sections

In [90]:
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np
from requests import get
from geopy import distance

In [327]:
# Create a list of zipcodes to scrape
blue_line.reset_index(drop=True, inplace=True)
blue_line.head()

,STOP_ID,STATION_NAME,STATION_DESCRIPTIVE_NAME,Location,Latitude,Longitude,Zipcode
0,30002,Austin,Austin (Blue Line),"(41.870851, -87.776812)",41.870851,-87.776812,60130
1,30013,Belmont,Belmont (Blue Line),"(41.938132, -87.712359)",41.938132,-87.712359,60659
2,30188,Cicero,Cicero (Blue Line),"(41.871574, -87.745154)",41.871574,-87.745154,60644
3,30374,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...","(41.885737, -87.630886)",41.885737,-87.630886,60696
4,30085,Clinton,Clinton (Blue Line),"(41.875539, -87.640984)",41.875539,-87.640984,60606


In [24]:
ziplist = blue_line['Zipcode']
ziplist = list(dict.fromkeys(ziplist))
print(ziplist)
print(str(len(ziplist)) + " unique zipcodes")

['60130', '60659', '60644', '60696', '60606', '60631', '60647', '60642', '60661', '60656', '60607', '60641', '60604', '60630', '60605', '60603', '60688', '60018', '60602', '60625', '60618']
21 unique zipcodes


In [26]:
# This function concats all of the search paramters into a long url
def get_cl_url (zipcode, distance, page, min_bed, max_bed):
    cl_url =("https://chicago.craigslist.org/search/apa?"
                   + "&search_distance="
                   + distance
                   + "&postal="
                   + zipcode
                   + "&min_bedrooms="
                   + min_bed
                   + "&max_bedrooms="
                   + max_bed
                   + "&s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
                   + "&availabilityMode=0")
    return cl_url

In [150]:
get_cl_url('60642', '2', '0', '0', '1')

'https://chicago.craigslist.org/search/apa?&search_distance=2&postal=60642&min_bedrooms=0&max_bedrooms=1&s=0&availabilityMode=0'

In [31]:
# This function pulls the location data by scraping the page on a URL
def get_loc_from_listing (url):
  #get the latitude and longitude from a URL listing
  listingHtml = get(url)
  listSoup = BeautifulSoup(listingHtml.text, 'html.parser')
  location = listSoup.find_all(class_= 'viewposting')
  loc = []
  if location[0] is None:
      loc.append('null')
      loc.append('null')

  else:
      loc.append(float(location[0]['data-latitude']))
      loc.append(float(location[0]['data-longitude']))
  return loc

In [143]:
# Create empty arrays for storing data
post_title_texts = []
post_hoods = []
post_links = []
post_prices = []
longitudes = []
latitudes = []
post_zipcodes = []

for zipcode in ziplist:
    
    #get the first page of the chicago housing listings in this zipcode
    response = get(get_cl_url(zipcode,'2','0','0','1')) # 0 or 1 bedrooms 2 mi from zipcode

    html_soup = BeautifulSoup(response.text, 'html.parser')

    #get the macro-container for the housing posts
    posts = html_soup.find_all('li', class_= 'result-row')


    #find the total number of posts to find the limit of the pagination
    results_num = html_soup.find('div', class_= 'search-legend')
    results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

    #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
    pages = np.arange(0, results_total+1, 120)

    iterations = 0


    for page in pages:

        #get request
        response = get(get_cl_url(zipcode,'2', page,'0','1')) #Searches for one bedrooms or studios

        sleep(randint(1,5))

        #throw warning for status codes that are not 200
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        #define the html text
        page_html = BeautifulSoup(response.text, 'html.parser')

        #define the posts
        posts = html_soup.find_all('li', class_= 'result-row')

        #extract data item-wise
        for post in posts:

            if post.find('span', class_ = 'result-hood') is not None:

                #neighborhoods
                post_hood = post.find('span', class_= 'result-hood').text
                post_hoods.append(post_hood)

                #title text
                post_title = post.find('a', class_='result-title hdrlnk')
                post_title_text = post_title.text
                post_title_texts.append(post_title_text)

                #post link
                post_link = post_title['href']
                post_links.append(post_link)

                # Location Data
                bothloc = get_loc_from_listing(post_link)
                latitudes.append(bothloc[0])
                longitudes.append(bothloc[1])

                #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
                post_price = post.a.text.strip().replace("$", "")
                post_prices.append(post_price)
                
                # Records the zipcode it was searched by
                post_zipcodes.append(zipcode)


        iterations += 1
        print("Page " + str(iterations) + " of zipcode "+ zipcode + " scraped successfully!")
    break

print("\n")

print("Scrape complete!")

Page 1 of zipcode 60130 scraped successfully!


Scrape complete!


In [141]:
chi_apts = pd.DataFrame({'neighborhood': post_hoods,
                       'post title': post_title_texts,
                        'URL': post_links,
                       'price': post_prices,
                        'longitude': longitudes,
                        'latitude': latitudes})
chi_apts.head(10)

ValueError: arrays must all be same length

In [142]:
chi_apts.shape

(48, 8)

In [38]:
# Drops any apartments missing price data
chi_apts.drop( chi_apts[ chi_apts['price'] == '' ].index , inplace=True)

Data Cleaning of all apartments dataframe


In [40]:

# Convert prices to int
chi_apts['price'] = chi_apts['price'].apply(lambda x: int(x))

In [41]:
# Drop any apartments less than 300 (often spam)
chi_apts.drop( chi_apts[ chi_apts['price'] <300 ].index , inplace=True)

In [43]:
# Remove duplicates with same URL - these are relistings
chi_apts = chi_apts.drop_duplicates(subset='URL')

In [45]:
chi_apts.sort_values(['price'], inplace=True)



chi_apts.reset_index(drop=True, inplace=True)

chi_apts.head(30)

,neighborhood,post title,URL,price,longitude,latitude
0,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400
1,(Oak Park),Great Clean 1 Bedroom ID: 69934000,https://chicago.craigslist.org/wcl/apa/d/oak-p...,795,-87.774557,41.880031
2,(Maywood IL),$850/mo- 1BR/ 1BA htd Maywood,https://chicago.craigslist.org/wcl/apa/d/maywo...,850,-87.843300,41.879300
3,(623 Harlem Ave),Forest Park Apartment for Rent,https://chicago.craigslist.org/wcl/apa/d/fores...,850,-87.804472,41.875073
4,(Berwyn),"Studios, one and two bedrooms available from $...",https://chicago.craigslist.org/wcl/apa/d/oak-p...,895,-87.781457,41.865184
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131
6,(Madison and Lathrop),Beautiful Downtown Forest Park Apartment,https://chicago.craigslist.org/wcl/apa/d/beaut...,925,-87.814358,41.879331
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131
8,(Washington and Kenilworth),One Bedroom Apartment in Oak Park,https://chicago.craigslist.org/wcl/apa/d/oak-p...,950,-87.803958,41.881531
9,(Oak Park),VINTAGE ONE BEDROOMS FROM $975- - 458,https://chicago.craigslist.org/wcl/apa/d/oak-p...,975,-87.786608,41.881824


In [75]:
chi_apts.shape

(48, 6)

In [147]:
# Remove duplicates with same post titles and price - these are most likely relistings
chi_apts = chi_apts.drop_duplicates(subset=['post title', 'price'])

In [149]:
chi_apts

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Neary Stations
0,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
3,(623 Harlem Ave),Forest Park Apartment for Rent,https://chicago.craigslist.org/wcl/apa/d/fores...,850,-87.804472,41.875073,Harlem (FB Branch),Harlem (Blue Line - Forest Park Branch)Harlem ...
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin,Austin Austin Austin Austin
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin,Austin Austin Austin Austin
10,(Forest Park),"FOREST PARK / OAK PARK 1 Bedroom, Balcony, 601...",https://chicago.craigslist.org/nch/apa/d/fores...,985,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
12,(Oak Park),GREAT LOCATION ONE BEDROOM FROM $995-1050- 950,https://chicago.craigslist.org/wcl/apa/d/oak-p...,995,-87.787700,41.872500,Oak Park,Oak Park Oak Park Oak Park Oak Park
15,(Forest Park),SEMI-MODERN ONE BEDROOMS FROM $1005-1015 - 227,https://chicago.craigslist.org/wcl/apa/d/fores...,1005,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
16,(7535 Jackson Blvd),Forest Park 1 Bedroom Apt Near Transportation!,https://chicago.craigslist.org/chc/apa/d/fores...,1040,-87.814258,41.875831,Forest Park,Forest Park Forest Park Forest Park Forest Park
18,(7535 Jackson Blvd),Forest Park 1 Bedroom Apt Near Transportation!,https://chicago.craigslist.org/chc/apa/d/fores...,1050,-87.814258,41.875831,Forest Park,Forest Park Forest Park Forest Park Forest Park
21,(Oak Park),ONE BEDROOM BLOCKS FROM 290! - 901,https://chicago.craigslist.org/wcl/apa/d/oak-p...,1095,-87.797958,41.872431,Oak Park,Oak Park Oak Park Oak Park Oak Park


In [435]:
chi_apts.shape

(106, 6)

In [76]:
for index, row in blue_line.iterrows(): 
    print(row['Station'])

Austin 
Belmont 
Cicero 
Clark/Lake (Blue, Brown, Green, Orange, Purple & Pink Lines)
Clinton 
Cumberland 
Damen 
Division 
Forest Park 
Grand 
Harlem (Blue Line - O'Hare Branch)
Harlem (Blue Line - Forest Park Branch)
Illinois Medical District 
Irving Park 
Jackson 
Jefferson Park 
Kedzie-Homan 
LaSalle 
Logan Square 
Montrose 
Monroe 
Oak Park 
Pulaski 
Racine 
O'Hare 
Rosemont 
UIC-Halsted 
Washington 
California 
Chicago 
Western (Blue Line - O'Hare Branch)
Western (Blue Line - Forest Park Branch)
Addison 


In [137]:
chi_apts['Nearby Stations'] = 'None'

In [123]:
blue_line.head()

,Station,Location,Latitude,Longitude
STOP_ID,,,,
30002,Austin,"(41.870851, -87.776812)",41.870851,-87.776812
30013,Belmont,"(41.938132, -87.712359)",41.938132,-87.712359
30188,Cicero,"(41.871574, -87.745154)",41.871574,-87.745154
30374,Clark/Lake,"(41.885737, -87.630886)",41.885737,-87.630886
30085,Clinton,"(41.875539, -87.640984)",41.875539,-87.640984


In [139]:
for index, row in blue_line.iterrows(): 
    station_lat = row['Latitude']
    station_long = row['Longitude']
    station_name = row['Station']
    for index, apt_row in chi_apts.iterrows():
        if distance.distance((station_lat, station_long), (apt_row.latitude, apt_row.longitude)).m < 600:
            #print("found one")
            #print(apt_row['Nearby Stations'])
            if chi_apts.at[index,'Nearby Stations'] == 'None':
                chi_apts.at[index,'Nearby Stations'] = station_name
            else:
                chi_apts.at[index,'Nearby Stations'] += (", " + station_name)
   #print(str(station_lat) + " " + str(station_long))

In [152]:
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Neary Stations
0,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
3,(623 Harlem Ave),Forest Park Apartment for Rent,https://chicago.craigslist.org/wcl/apa/d/fores...,850,-87.804472,41.875073,Harlem (FB Branch),Harlem (Blue Line - Forest Park Branch)Harlem ...
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin,Austin Austin Austin Austin
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin,Austin Austin Austin Austin
10,(Forest Park),"FOREST PARK / OAK PARK 1 Bedroom, Balcony, 601...",https://chicago.craigslist.org/nch/apa/d/fores...,985,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...


In [144]:
# Drops any apartments not close to a train
chi_apts.drop( chi_apts[ chi_apts['Nearby Stations'] == 'None' ].index , inplace=True)

In [153]:
chi_apts

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Neary Stations
0,(Elmhurst),CHEAP & CLEAN 1 BDRM GARDEN APT.,https://chicago.craigslist.org/wcl/apa/d/fores...,750,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
3,(623 Harlem Ave),Forest Park Apartment for Rent,https://chicago.craigslist.org/wcl/apa/d/fores...,850,-87.804472,41.875073,Harlem (FB Branch),Harlem (Blue Line - Forest Park Branch)Harlem ...
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin,Austin Austin Austin Austin
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin,Austin Austin Austin Austin
10,(Forest Park),"FOREST PARK / OAK PARK 1 Bedroom, Balcony, 601...",https://chicago.craigslist.org/nch/apa/d/fores...,985,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
12,(Oak Park),GREAT LOCATION ONE BEDROOM FROM $995-1050- 950,https://chicago.craigslist.org/wcl/apa/d/oak-p...,995,-87.787700,41.872500,Oak Park,Oak Park Oak Park Oak Park Oak Park
15,(Forest Park),SEMI-MODERN ONE BEDROOMS FROM $1005-1015 - 227,https://chicago.craigslist.org/wcl/apa/d/fores...,1005,-87.810600,41.874400,"Forest Park , Harlem (FB Branch)",Harlem (Blue Line - Forest Park Branch)Forest ...
16,(7535 Jackson Blvd),Forest Park 1 Bedroom Apt Near Transportation!,https://chicago.craigslist.org/chc/apa/d/fores...,1040,-87.814258,41.875831,Forest Park,Forest Park Forest Park Forest Park Forest Park
18,(7535 Jackson Blvd),Forest Park 1 Bedroom Apt Near Transportation!,https://chicago.craigslist.org/chc/apa/d/fores...,1050,-87.814258,41.875831,Forest Park,Forest Park Forest Park Forest Park Forest Park
21,(Oak Park),ONE BEDROOM BLOCKS FROM 290! - 901,https://chicago.craigslist.org/wcl/apa/d/oak-p...,1095,-87.797958,41.872431,Oak Park,Oak Park Oak Park Oak Park Oak Park


In [155]:
austin = chi_apts[chi_apts['Nearby Stations'].str.contains("Austin")]
austin.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Neary Stations
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin,Austin Austin Austin Austin
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin,Austin Austin Austin Austin
62,(Oak Park),VINTAGE ONE BEDROOM - 900,https://chicago.craigslist.org/wcl/apa/d/oak-p...,1450,-87.777957,41.872658,Austin,Austin Austin Austin Austin


In [160]:
austin.mean().price

1091.6666666666667

In [165]:
# Rates the price of the apartment
chi_apts['Rent'] = pd.cut(chi_apts['price'], bins=3, labels=['low', 'med', 'high'])

In [168]:
chi_apts['Rent'].value_counts()

high    8
med     8
low     4
Name: Rent, dtype: int64

In [164]:
test = chi_apts.loc[chi_apts['column_name'].str.contains('Austin')]

0     low
3     low
5     low
7     low
10    med
Name: price, dtype: category
Categories (3, object): [low < med < high]

In [169]:
austin = chi_apts[chi_apts['Nearby Stations'].str.contains("Austin")]
austin.head()

,neighborhood,post title,URL,price,longitude,latitude,Nearby Stations,Neary Stations,Rent
5,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,900,-87.774157,41.869131,Austin,Austin Austin Austin Austin,low
7,(822 S. Austin),OAK PARK!Spacious Bright 1 Bedroom $925 Oak ...,https://chicago.craigslist.org/wcl/apa/d/oak-p...,925,-87.774157,41.869131,Austin,Austin Austin Austin Austin,low
62,(Oak Park),VINTAGE ONE BEDROOM - 900,https://chicago.craigslist.org/wcl/apa/d/oak-p...,1450,-87.777957,41.872658,Austin,Austin Austin Austin Austin,high


In [171]:
# Builds empty dataframe for storing all data on each train stop (our goal!)
columns = ['Train Stop', 'Avg Rent', 'Num Apts', 'Cheap', 'Med', 'Luxury', 'Bars']
stop_data = pd.DataFrame(columns=columns)

# Stores the names of each stop in a row
stop_data['Train Stop'] = blue_line['Station']

In [190]:
stop_data.head()

,Train Stop,Avg Rent,Num Apts,Cheap,Med,Luxury,Bars
STOP_ID,,,,,,,
30002,Austin,1091,NaN,NaN,NaN,NaN,NaN
30013,Belmont,1091,NaN,NaN,NaN,NaN,NaN
30188,Cicero,1091,NaN,NaN,NaN,NaN,NaN
30374,Clark/Lake,1091,NaN,NaN,NaN,NaN,NaN
30085,Clinton,1091,NaN,NaN,NaN,NaN,NaN


In [189]:
stop_data['Avg Rent'] = int(chi_apts[chi_apts['Nearby Stations'].str.contains('Austin')].mean().price)

In [186]:
stop_data['Train Stop']

STOP_ID
30002                       Austin 
30013                      Belmont 
30188                       Cicero 
30374                    Clark/Lake
30085                      Clinton 
30045                   Cumberland 
30116                        Damen 
30063                     Division 
30076                  Forest Park 
30096                        Grand 
30146              Harlem OH Branch
30189            Harlem (FB Branch)
30158    Illinois Medical District 
30108                  Irving Park 
30015                      Jackson 
30248               Jefferson Park 
30049                 Kedzie-Homan 
30262                      LaSalle 
30198                 Logan Square 
30260                     Montrose 
30154                       Monroe 
30035                     Oak Park 
30180                      Pulaski 
30093                       Racine 
30172                       O'Hare 
30160                     Rosemont 
30069                  UIC-Halsted 
30073               

In [27]:
chi_apts.iloc[0].URL

'https://chicago.craigslist.org/chc/apa/d/chicago-luxurious-1-bed-1-bath-condo-in/7065399594.html'

In [166]:
print(distance.distance((damen_lat, damen_long), (chi_apts.iloc[16].latitude, chi_apts.iloc[16].longitude)).m)

523.414145424127


In [251]:
chi_apts['Stop'] = 'Damen'
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop
0,(Chicago - Bucktown),Rehabbed Studio Loft! Granite/Stainless. Hardw...,https://chicago.craigslist.org/chc/apa/d/chica...,1650,-87.7043,41.9209,Damen
1,(Chicago - Lincoln Park),Great Value 1BR with WD in unit in Lincoln Park!,https://chicago.craigslist.org/chc/apa/d/chica...,1525,-87.7043,41.9209,Damen
2,(Dickens & Central Park),"Choice 1 Bedroom, 1 Bath in Logan Square, Avai...",https://chicago.craigslist.org/chc/apa/d/chica...,1350,-87.7043,41.9209,Damen
3,(Logan Square),Logan Square 1 bedroom heat included,https://chicago.craigslist.org/chc/apa/d/chica...,1095,-87.7043,41.9209,Damen
4,(Chicago),"Cute, Logan Square, free laundry, central air,...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.7043,41.9209,Damen


In [252]:
def distance_to_train (stop_lat, stop_long, train_lat = damen_lat, train_long = damen_long):
    return distance.distance((train_lat, train_long), (stop_lat, stop_long)).m

In [174]:
chi_apts.iloc[1].latitude

41.9019

In [253]:
chi_apts['Distance to train'] = chi_apts.apply(lambda x: distance_to_train(x.latitude, x.longitude), axis=1)

In [254]:
chi_apts.head(30)

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Chicago - Bucktown),Rehabbed Studio Loft! Granite/Stainless. Hardw...,https://chicago.craigslist.org/chc/apa/d/chica...,1650,-87.704300,41.920900,Damen,2549.886832
1,(Chicago - Lincoln Park),Great Value 1BR with WD in unit in Lincoln Park!,https://chicago.craigslist.org/chc/apa/d/chica...,1525,-87.704300,41.920900,Damen,2549.886832
2,(Dickens & Central Park),"Choice 1 Bedroom, 1 Bath in Logan Square, Avai...",https://chicago.craigslist.org/chc/apa/d/chica...,1350,-87.704300,41.920900,Damen,2549.886832
3,(Logan Square),Logan Square 1 bedroom heat included,https://chicago.craigslist.org/chc/apa/d/chica...,1095,-87.704300,41.920900,Damen,2549.886832
4,(Chicago),"Cute, Logan Square, free laundry, central air,...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.704300,41.920900,Damen,2549.886832
5,(Chicago),Extremely spacious one bedroom with one bathro...,https://chicago.craigslist.org/chc/apa/d/chica...,1499,-87.704300,41.920900,Damen,2549.886832
6,(Wicker Park),1.5 MONTHS FREE on this 1 bed 1 bath Blue Line...,https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.704300,41.920900,Damen,2549.886832
7,(Humboldt Park),Affordable Housing for residents with limited ...,https://chicago.craigslist.org/chc/apa/d/chica...,897,-87.704300,41.920900,Damen,2549.886832
8,(Lakeview),Luxury One Bedroom,https://chicago.craigslist.org/chc/apa/d/chica...,2350,-87.704300,41.920900,Damen,2549.886832
9,(Chicago),Bucktown- Armitage/ Damen,https://chicago.craigslist.org/chc/apa/d/chica...,1150,-87.704300,41.920900,Damen,2549.886832


In [255]:
chi_apts.sort_values(['Distance to train'], inplace=True)
chi_apts.reset_index(drop=True, inplace=True)
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
1,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
2,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
3,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
4,(Wicker Park),Live in the heart of Wicker Park / 2 bedrooms ...,https://chicago.craigslist.org/chc/apa/d/chica...,1066,-87.677554,41.910532,Damen,88.061029


In [256]:
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
4,(Wicker Park),Live in the heart of Wicker Park / 2 bedrooms ...,https://chicago.craigslist.org/chc/apa/d/chica...,1066,-87.677554,41.910532,Damen,88.061029
8,(Wicker Park),"Junior 1-bedroom in heart of Wicker, steps to ...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.677554,41.911027,Damen,142.835023
12,"(6 Corners, Wicker Park, Chicago)","Wicker Park 1 Bed, Utilities Included - Fully ...",https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.674854,41.908632,Damen,247.351596
20,(Wicker Park),Large 1.5 bedroom in Wicker Park,https://chicago.craigslist.org/chc/apa/d/chica...,1930,-87.680009,41.911232,Damen,269.909557


In [257]:
chi_apts.shape

(91, 8)

In [258]:
chi_apts.sort_values(['Distance to train'], inplace=True)
chi_apts.reset_index(drop=True, inplace=True)
chi_apts.head()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
0,(Wicker Park),"Furnished Urban Wicker Park 1BR w/ W/D, nr. Da...",https://chicago.craigslist.org/chc/apa/d/chica...,2790,-87.676854,41.910007,Damen,56.506599
1,(Wicker Park),Live in the heart of Wicker Park / 2 bedrooms ...,https://chicago.craigslist.org/chc/apa/d/chica...,1066,-87.677554,41.910532,Damen,88.061029
2,(Wicker Park),"Junior 1-bedroom in heart of Wicker, steps to ...",https://chicago.craigslist.org/chc/apa/d/chica...,1100,-87.677554,41.911027,Damen,142.835023
3,"(6 Corners, Wicker Park, Chicago)","Wicker Park 1 Bed, Utilities Included - Fully ...",https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.674854,41.908632,Damen,247.351596
4,(Wicker Park),Large 1.5 bedroom in Wicker Park,https://chicago.craigslist.org/chc/apa/d/chica...,1930,-87.680009,41.911232,Damen,269.909557


In [259]:
chi_apts.drop( chi_apts[ chi_apts['Distance to train'] >600 ].index , inplace=True)
chi_apts.tail()

,neighborhood,post title,URL,price,longitude,latitude,Stop,Distance to train
6,(Chicago),*Gut Rehabbed* 1BED+DEN/1BTH In the Heart of B...,https://chicago.craigslist.org/chc/apa/d/chica...,2100,-87.680566,41.911932,Damen,355.603617
7,(WOLCOTT AVENUE),***NEW REDUCED PRICE*** HEART OF BUCKTOWN-ONE ...,https://chicago.craigslist.org/chc/apa/d/chica...,1550,-87.675254,41.912993,Damen,403.770377
8,(Chicago),"Bucktown - Spacious 1 B.R., Extra Large Kitche...",https://chicago.craigslist.org/chc/apa/d/chica...,1900,-87.673183,41.912332,Damen,455.186005
9,(2007 W. Potomac- #2),Wicker Park Apartment with EVERYTHING! (2007-#2),https://chicago.craigslist.org/chc/apa/d/chica...,1975,-87.677354,41.905032,Damen,523.414145
10,(CHICAGO),BUCKTOWN 1 BED TIMBER LOFT- PARKING INCLUDED- ...,https://chicago.craigslist.org/chc/apa/d/buckt...,2200,-87.680155,41.914732,Damen,598.157905


In [260]:
chi_apts.shape

(11, 8)

In [201]:
chi_apts.iloc[11].URL

'https://chicago.craigslist.org/chc/apa/d/chicago-1-month-free-no-deposit-best-of/7077312182.html'

In [261]:
# create map of Chciago using latitude and longitude values
map_damen = folium.Map(location=[damen_lat, damen_long], zoom_start=15)

# add station markers to map
for lat, lng, station in zip(blue_line['Latitude'], blue_line['Longitude'], blue_line['STATION_NAME']):
    label = station
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_damen)
    folium.Circle([lat, lng], radius=600, color='red', fill=False).add_to(map_damen)

# add apartment markers to map
for lat, lng, price in zip(chi_apts['latitude'], chi_apts['longitude'], chi_apts['price']):
    label = price
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        parse_html=False).add_to(map_damen)
folium.Circle([lat, lng], radius=600, color='red', fill=False).add_to(map_damen)  
    
map_damen

The geograpical coordinate of Chicago are 41.8925325, -87.6576948.
